In [1]:
from ftplib import FTP
import copy

import numpy as np

from astropy.coordinates import Angle
from astropy.time import Time, TimeDelta
from astropy import units as u
# для astropy в режиме offline
from astropy.utils import iers
iers.conf.auto_download = True 

from secret_key import login, password

In [56]:
"""
N1 – 194.149.67.97 (от 20° до 42°)
N2 – 194.149.67.38 (от -9° до 20°)
"""

con = FTP('194.149.67.38', login, password)

In [57]:
file_list = con.nlst('*.pnt')

In [58]:
def read_pnt_header(line):
    header_pnt.append(line.decode("utf-8"))

In [61]:
pulsar_ra = Angle("11:36:03.2477", unit='hourangle')
file_with_pulsar = []
for file in file_list:
    con = FTP('194.149.67.38', login, password)
    header_pnt = []
    try:
        con.retrbinary('RETR ' + file, callback=read_pnt_header, blocksize=493)
    except UnicodeDecodeError:
        print('Read was done ' + file + '!')
    except ConnectionResetError:
        print('!!! Connection was aborted !!!')
        
    header = {}
    for line in data[0].split('\n'):
        a, *b = line.split()
        header[a] = b
        
    t_b, t_e = get_time_begin_and_end(header)
    sid_t_b = my_sidereal_time(t_b)
    sid_t_e = my_sidereal_time(t_e - 4*u.hour)
    
    if sid_t_b < pulsar_ra < sid_t_e:
        file_with_pulsar.append(file)

Read was done 010419_00_N2_00.pnt!
Read was done 010419_01_N2_00.pnt!
Read was done 010419_02_N2_00.pnt!
Read was done 010419_03_N2_00.pnt!
Read was done 010419_04_N2_00.pnt!
Read was done 010419_05_N2_00.pnt!
Read was done 010419_06_N2_00.pnt!
Read was done 010419_07_N2_00.pnt!
Read was done 010419_08_N2_00.pnt!
Read was done 010419_09_N2_00.pnt!
Read was done 010419_10_N2_00.pnt!
Read was done 010419_11_N2_00.pnt!
Read was done 010419_12_N2_00.pnt!
Read was done 010419_13_N2_00.pnt!
Read was done 010419_14_N2_00.pnt!
Read was done 010419_15_N2_00.pnt!
Read was done 010419_16_N2_00.pnt!
Read was done 010419_17_N2_00.pnt!
Read was done 010419_18_N2_00.pnt!
Read was done 010419_19_N2_00.pnt!
Read was done 010419_20_N2_00.pnt!
Read was done 010419_21_N2_00.pnt!
Read was done 010419_22_N2_00.pnt!
Read was done 010419_23_N2_00.pnt!
Read was done 020419_00_N2_00.pnt!
Read was done 020419_01_N2_00.pnt!
Read was done 020419_02_N2_00.pnt!
Read was done 020419_03_N2_00.pnt!
Read was done 020419

Read was done 100419_19_N2_00.pnt!
Read was done 100419_20_N2_00.pnt!
Read was done 100419_21_N2_00.pnt!
Read was done 100419_22_N2_00.pnt!
Read was done 100419_23_N2_00.pnt!
Read was done 110419_00_N2_00.pnt!
Read was done 110419_01_N2_00.pnt!
Read was done 110419_02_N2_00.pnt!
Read was done 110419_03_N2_00.pnt!
Read was done 110419_04_N2_00.pnt!
Read was done 110419_05_N2_00.pnt!
Read was done 110419_06_N2_00.pnt!
Read was done 110419_07_N2_00.pnt!
Read was done 110419_08_N2_00.pnt!
Read was done 110419_09_N2_00.pnt!
Read was done 110419_10_N2_00.pnt!
Read was done 110419_11_N2_00.pnt!
Read was done 110419_12_N2_00.pnt!
Read was done 110419_13_N2_00.pnt!
Read was done 110419_14_N2_00.pnt!
Read was done 110419_15_N2_00.pnt!
Read was done 110419_16_N2_00.pnt!
Read was done 110419_17_N2_00.pnt!
Read was done 110419_18_N2_00.pnt!
Read was done 110419_19_N2_00.pnt!
Read was done 110419_20_N2_00.pnt!
Read was done 110419_21_N2_00.pnt!
Read was done 110419_22_N2_00.pnt!
Read was done 110419

Read was done 200419_16_N2_00.pnt!
Read was done 200419_17_N2_00.pnt!
Read was done 200419_18_N2_00.pnt!
Read was done 200419_19_N2_00.pnt!
Read was done 200419_20_N2_00.pnt!
Read was done 200419_21_N2_00.pnt!
Read was done 200419_22_N2_00.pnt!
Read was done 200419_23_N2_00.pnt!
Read was done 210419_00_N2_00.pnt!
Read was done 210419_01_N2_00.pnt!
Read was done 210419_02_N2_00.pnt!
Read was done 210419_03_N2_00.pnt!
Read was done 210419_04_N2_00.pnt!
Read was done 210419_05_N2_00.pnt!
Read was done 210419_06_N2_00.pnt!
Read was done 210419_07_N2_00.pnt!
Read was done 210419_08_N2_00.pnt!
Read was done 210419_09_N2_00.pnt!
Read was done 210419_10_N2_00.pnt!
Read was done 210419_11_N2_00.pnt!
Read was done 210419_12_N2_00.pnt!
Read was done 210419_13_N2_00.pnt!
Read was done 210419_14_N2_00.pnt!
Read was done 210419_15_N2_00.pnt!
Read was done 210419_16_N2_00.pnt!
Read was done 210419_17_N2_00.pnt!
Read was done 210419_18_N2_00.pnt!


In [62]:
len(file_with_pulsar)

21

In [64]:
file_with_pulsar[1]

'020419_01_N2_00.pnt'

In [65]:
 header

{'alpha': ['alpha'],
 'date_begin': ['21.04.2019', 'UTC', '21.04.2019'],
 'date_end': ['21.04.2019'],
 'delta': ['delta'],
 'fbands': ['109.20751953125',
  '109.62255859375',
  '110.03759765625',
  '110.45263671875',
  '110.86767578125',
  '111.287597656'],
 'fcentral': ['110.25'],
 'modulus': ['1', '2', '3', '4', '5', '6'],
 'nbands': ['6'],
 'npoints': ['36017'],
 'numpar': ['16'],
 'source': ['source'],
 'time_begin': ['17:00:00', 'UTC', '13:00:00'],
 'time_end': ['17:59:59'],
 'tresolution': ['99.9424'],
 'wb_total': ['2.5'],
 'wbands': ['0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4248046875']}

In [71]:
my_sidereal_time(t_b)

<Angle 5.50369477 hourangle>

In [70]:
t_b

<Time object: scale='utc' format='isot' value=2019-04-21T13:00:00.0000000>